In [ ]:
!pip install jupyter_dash

In [16]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import plotly.express as px
from dash import no_update

In [17]:
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

In [18]:
JupyterDash.infer_jupyter_proxy_config()

In [20]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2021-09-20 21:23:08--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv.1’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-09-20 21:23:09 (556 MB/s) - ‘spacex_launch_dash.csv.1’ saved [3033/3033]



In [35]:


# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
# app = dash.Dash(__name__)
app = JupyterDash(__name__)


# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site_dropdown',
                                             options=[
                                                    {'label': 'All Sites', 'value': 'All Sites'},
                                                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                             ],
                                             placeholder='Select a Launch Site here', 
                                             searchable = True , 
                                             value = 'All Sites'),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(
                                    id='payload_slider',
                                    min=0, max=10000, step=1000,  
                                    marks = {
                                            0: '0 kg',
                                            1000: '1000 kg',
                                            2000: '2000 kg',
                                            3000: '3000 kg',
                                            4000: '4000 kg',
                                            5000: '5000 kg',
                                            6000: '6000 kg',  
                                            7000: '7000 kg',
                                            8000: '8000 kg',
                                            9000: '9000 kg',
                                            10000: '10000 kg'                                         
                                    },
                                    value=[min_payload,max_payload]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
     Output(component_id='success-pie-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value')]               
)
def pie_plot(site_dropdown):
    if (site_dropdown == 'All Sites'):
        data  = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(data, names = 'Launch Site',title = 'Total of Success Launches')
    else:
        data  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(data, names = 'class',title = 'Total of Success Launches')
    return fig

@app.callback(
     Output(component_id='success-payload-scatter-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value'),Input(component_id="payload_slider", component_property="value")]               
)
def scatter_plot(site_dropdown,payload_slider):
    if site_dropdown == 'All Sites':
        low, high = payload_slider
        data  = spacex_df
        mask = (data['Payload Mass (kg)'] > low) & (data['Payload Mass (kg)'] < high)
        fig = px.scatter(data[mask], x="Payload Mass (kg)", y="class",  color ="Booster Version",size='Payload Mass (kg)',hover_data=['Payload Mass (kg)'],template='plotly_white')
    else:
        low, high = payload_slider
        data  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        mask = (data['Payload Mass (kg)'] > low) & (data['Payload Mass (kg)'] < high)
        fig = px.scatter(data[mask], x="Payload Mass (kg)", y="class",  color="Booster Version", size='Payload Mass (kg)',hover_data=['Payload Mass (kg)'],template='plotly_white')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False, mode='inline')

127.0.0.1 - - [20/Sep/2021 22:06:15] "GET /_shutdown_20ba2f6c-cba0-4c6a-a7f9-20bb5657d44d HTTP/1.1" 200 -
 * Running on http://localhost:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Sep/2021 22:06:15] "GET /_alive_20ba2f6c-cba0-4c6a-a7f9-20bb5657d44d HTTP/1.1" 200 -


<IPython.core.display.Javascript object>